# **Fast Forward Neural Network** (FFNN) Implementation
#### _with **Forward Propagation**_

#### Made by:
- Samuel Christoper Swandi - 13520075
- Grace Claudia - 13520078
- Ubaidillah Ariq Prathama - 13520085
- Patrick Amadeus Irawan - 13520109

------------

List of Content

1. [Library & Dependencies](#library)
2. [Helper Function](#helper)
3. [Neural Network Visualization](#visualization)

## Library & Dependencies <a name="library"></a>

In [ ]:
!pip install pyvis==0.3.2
!pip install networkx==2.6.3
!pip install numpy==1.21.6

In [11]:
import numpy as np
import os

from pyvis.network import Network
import networkx as nx

## **Helper** Function <a class="anchor" id="helper"></a>

#### Activation Function

In [ ]:
def linear(net):
    return net

def ReLU(net):
    return np.maximum(0,net)

def sigmoid(net):
    return 1/(1+np.exp(-net))

def softmax(net):
    return np.exp(net)/np.sum(np.exp(net))

#### Loss Function

In [ ]:
def mse(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))

def sse(y_true, y_pred):
    return np.sum(np.square(y_true - y_pred))

## Neural Network **Visualization** <a class="anchor" id="visualization"></a>

In [ ]:
g = Network(notebook=True, cdn_resources="remote")
nxg = nx.complete_graph(5)
g.from_nx(nxg)
g.show("./tmp/network.html")